### Model without any flexibility

In [2]:
library(statsecol)
library(ggplot2)
data(wildebeest)

Warning message:
“package ‘ggplot2’ was built under R version 4.1.1”


In [3]:
logistic_nll <- function(pars,years,removals, Nhat, SEhat){
  #parameter set up
  N0 <- exp(pars[1])
  r <- exp(pars[2])
  k <- exp(pars[3])
  N <- numeric(years)
  N[1] <- N0
  
  #generate population dynamics:
  for(i in 2:years){
    N[i]=N[i-1] + r * N[i-1] * (1 - N[i-1] / k) - removals[i-1]
  }
  
  #calculate the NEGATIVE LOG likelihood
  # - calculate the Log Lik for each value (dnorm())
  # - sum them up (sum(dnorm()))
  # - take the negative of the sum of the Log Likelihoods (-sum(dnorm()))
  
  negloglik <- -sum(dnorm(Nhat,N,SEhat, log=TRUE), na.rm=TRUE)
  
  #return the negative log likelihood 
  return(negloglik)
}

#use guesses as starting values
N0 <- log(0.1)
r <- log(0.25)
k <-  log(1.5)
pars <- c(N0,r,k)
logistic_fit <- optim(
    par = pars,                 #initial parameter vector
    fn = logistic_nll,          #the function to me minimizes
    years = nrow(wildebeest),   #\
    removals = wildebeest$Catch,# \
    Nhat = wildebeest$Nhat,     #  \
    SEhat = wildebeest$sehat)   #   \->arguments passed to 'fn'

### Model with carrying capacity K set as an exponential function of rain

In [4]:
rainK_nll <- function(pars, years, removals, Nhat, SEhat, rain){
  #parameter set up
  N0 <- exp(pars[1])
  r <- exp(pars[2])
  beta0 <- pars[3]   #not transformed |
  beta1 <- pars[4]   #not transformed |-> note extra parameter now
  N <- numeric(years)
  k <- numeric(years)
  N[1] <- N0
  k[1] <- NA #1st K not in the model 
  
  #generate population dynamics:
  for(i in 2:years){
    k[i] <- exp(beta0 + beta1*rain[i]) #link fn of the linear predictor 
    N[i]=N[i-1] + r * N[i-1] * (1-N[i-1]/k[i]) - removals[i-1]
  }
  
  #calculate the NEGATIVE LOG likelihood
  # - calculate the Log Lik for each value (dnorm())
  # - sum them up (sum(dnorm()))
  # - take the negative of the sum of the Log Likelihoods (-sum(dnorm()))
  
  negloglik <- -sum(dnorm(Nhat,N,SEhat, log=TRUE), na.rm=TRUE)
  
  #return the negative log likelihood 
  return(negloglik)
}

N0 <- log(0.1)
r <- log(0.25)
beta0 <-  log(1.5)   #previous value
beta1 <-  0          #no effect

pars_rain <- c(N0,r,beta0,beta1)
rain_fit <- optim(
    par = pars_rain,
    fn = rainK_nll,
    years = nrow(wildebeest), 
    removals = wildebeest$Catch,
    Nhat = wildebeest$Nhat,
    SEhat = wildebeest$sehat,
    rain = wildebeest$rain
)

### Model growth rate as a function of rain

In [32]:
growth_rate_nll <- function(pars, years, removals, Nhat, SEhat, rain){
  #parameter set up
  N0 <- exp(pars[1])
  K <- exp(pars[2])
  beta0 <- pars[3]   #not transformed |
  beta1 <- pars[4]   #not transformed |-> note extra parameter now
  N <- numeric(years)
  r <- numeric(years)
  N[1] <- N0
  r[1] <- NA #1st K not in the model 
  
  #generate population dynamics:
  for(i in 2:years){
    r[i] <- exp(beta0 + beta1*rain[i]) #link fn of the linear predictor 
    N[i]=N[i-1] + r[i] * N[i-1] * (1-N[i-1]/K) - removals[i-1]
  }
  
  #calculate the NEGATIVE LOG likelihood
  # - calculate the Log Lik for each value (dnorm())
  # - sum them up (sum(dnorm()))
  # - take the negative of the sum of the Log Likelihoods (-sum(dnorm()))
  
  negloglik <- -sum(dnorm(Nhat,N,SEhat, log=TRUE), na.rm=TRUE)
  
  #return the negative log likelihood 
  return(negloglik)
}

N0 <- log(0.1)
K <-  log(1.5)
beta0 <-  log(1.5)   #previous value
beta1 <-  0          #no effect

pars_growth <- c(N0,K,beta0,beta1)
growth_rate_fit <- optim(
    par = pars_rain,
    fn = growth_rate_nll,
    years = nrow(wildebeest), 
    removals = wildebeest$Catch,
    Nhat = wildebeest$Nhat,
    SEhat = wildebeest$sehat,
    rain = wildebeest$rain
)



### Model both carrying capacity and growth rate as a function of rain

In [41]:
multiple_nll <- function(pars, years, removals, Nhat, SEhat, rain){
  #parameter set up
  N0 <- exp(pars[1])
  a0 <- pars[2]   #not transformed |
  a1 <- pars[3]   #not transformed |-> note extra parameter now
  b0 <- pars[4]
  b1 <- pars[5]
  N <- numeric(years)
  k <- numeric(years)
  r <- numeric(years)
  
  N[1] <- N0
  r[1] <- NA #1st K not in the model 
  k[1] <- NA
    
  #generate population dynamics:
  for(i in 2:years){
    r[i] <- exp(a0 + a1*rain[i]) #link fn of the linear predictor 
    k[i] <- exp(b0 + b1*rain[i]) 
    N[i]=N[i-1] + r[i] * N[i-1] * (1-N[i-1]/k[i]) - removals[i-1]
  }
  
  #calculate the NEGATIVE LOG likelihood
  # - calculate the Log Lik for each value (dnorm())
  # - sum them up (sum(dnorm()))
  # - take the negative of the sum of the Log Likelihoods (-sum(dnorm()))
  
  negloglik <- -sum(dnorm(Nhat,N,SEhat, log=TRUE), na.rm=TRUE)
  
  #return the negative log likelihood 
  return(negloglik)
}

N0 <- log(0.1)
a0 <-  log(1.5)   #previous value
a1 <-  0          #no effect
b0 <-  log(1.5)   #previous value
b1 <-  0          #no effect

pars_multiple <- c(N0, a0, a1, b0, b1)
multiple_fit <- optim(
    par = pars_multiple,
    fn = multiple_nll,
    years = nrow(wildebeest), 
    removals = wildebeest$Catch,
    Nhat = wildebeest$Nhat,
    SEhat = wildebeest$sehat,
    rain = wildebeest$rain
)


### AIC for the 4 models

In [42]:
logistic_K = length(pars)
logistic_AIC = 2 * logistic_K + 2 * logistic_fit$value

rain_K = length(pars_rain)
rain_AIC = 2 * rain_K + 2 * rain_fit$value

growth_K = length(pars_growth)
growth_AIC = 2 * growth_K + 2 * growth_rate_fit$value

multiple_K = length(pars_multiple)
multiple_AIC  = 2 * multiple_K + 2 * multiple_fit$value
 

In [43]:
data.frame(
    Model = c("logistic","rain", "growth", "multiple"),
    K = c(logistic_K, rain_K, growth_K, multiple_K),
    nll = c(logistic_fit$value, rain_fit$value, growth_rate_fit$value, multiple_fit$value),
    AIC = c(logistic_AIC, rain_AIC, growth_AIC, multiple_AIC),
    dAIC = c(logistic_AIC, rain_AIC, growth_AIC, multiple_AIC)- min(c(logistic_AIC, rain_AIC, growth_AIC, multiple_AIC)))

Model,K,nll,AIC,dAIC
<chr>,<int>,<dbl>,<dbl>,<dbl>
logistic,3,-12.54118,-19.08236,0.9042661
rain,4,-13.99331,-19.98662,0.0000000
growth,4,-11.15333,-14.30667,5.6799578
multiple,5,33.17592,76.35184,96.3384641


In [44]:
growth_rate_fit

$par
[1] -1.14920235  2.46956574 -2.67788790  0.06199493

$value
[1] -11.15333

$counts
function gradient 
     501       NA 

$convergence
[1] 1

$message
NULL